In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
import pandas as pd
import glob

In [2]:
autoencoder_model = tf.keras.saving.load_model('autoencoder_model')
autoencoder_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 128, 176, 1)]     0         
                                                                 
 conv2d_18 (Conv2D)          (None, 128, 176, 128)     1280      
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 64, 88, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 64, 88, 32)        36896     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 32, 44, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_20 (Conv2D)          (None, 16, 22, 8)         2312

In [3]:
# Create a new model that only includes the encoder part
encoder_model = tf.keras.Model(inputs=autoencoder_model.input, outputs=autoencoder_model.get_layer('dense_6').output)

In [4]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

def audio_to_mel_spectrogram(audio_file_path, max_length=4*22500, sr=22500, n_mels=256):
    # Load the audio file
    y, sr = librosa.load(audio_file_path, sr=sr)
    
    if len(y) > max_length:
        # If the audio is longer than the maximum length, cut it
        y = y[:max_length]
    elif len(y) < max_length:
        # If the audio is shorter, pad it with zeros
        pad_length = max_length - len(y)
        y = np.pad(y, (0, pad_length), mode='constant')
    
    # Calculate the Mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_mels)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
    
    return mel_spectrogram

# Generator function for lazy loading of audio data
def audio_data_generator(audio_files):
    for audio_file in audio_files:
        yield audio_to_mel_spectrogram(audio_file).reshape((1, 128, 176, 1))


In [5]:
tinit_train = pd.read_csv('tinit_train.csv')
tinit_test = pd.read_csv('tinit_test.csv')
wavefake_train = pd.read_csv('wavefake_train.csv')
wavefake_test = pd.read_csv('wavefake_test.csv')

In [ ]:
%%time
tinit_train_generator = tf.data.Dataset.from_generator(audio_data_generator, args=[tinit_train['full_path']], output_signature=tf.TensorSpec(shape=(1, 128, 176, 1)))
np.savetxt('tinit_train_x.csv', encoder_model.predict(tinit_train_generator), delimiter=",")

      5/Unknown - 9s 16ms/step

C:\Users\gabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


   2725/Unknown - 103s 35ms/step

In [ ]:
%%time
tinit_test_generator = tf.data.Dataset.from_generator(audio_data_generator, args=[tinit_test['full_path']], output_signature=tf.TensorSpec(shape=(1, 128, 176, 1)))
np.savetxt('tinit_test_x.csv', encoder_model.predict(tinit_test_generator), delimiter=",")

In [ ]:
%%time
wavefake_train_generator = tf.data.Dataset.from_generator(audio_data_generator, args=[wavefake_train['full_path']], output_signature=tf.TensorSpec(shape=(1, 128, 176, 1)))
np.savetxt('wavefake_train_x.csv', encoder_model.predict(wavefake_train_generator), delimiter=",")

In [ ]:
%%time
wavefake_test_generator = tf.data.Dataset.from_generator(audio_data_generator, args=[wavefake_test['full_path']], output_signature=tf.TensorSpec(shape=(1, 128, 176, 1)))
np.savetxt('wavefake_test_x.csv', encoder_model.predict(wavefake_test_generator), delimiter=",")